In [1]:
import pandas as pd
from functions import assembleDf, epochElo, epochsElo, epochG, epochsG,PlayerElo
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time

In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

In [3]:
singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_5709/3293468424.py:1: DtypeWarning: Columns (3,9,17) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches = pd.read_csv('../Data/singles_matches_df.csv',parse_dates = ['tourney_date'],


In [4]:
playerClasses, ratingsHistory = epochsElo(matches)

In [5]:
ratingsHistory_df = assembleDf(ratingsHistory)

In [6]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1878-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1879-07-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1880-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1881-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1882-07-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,NaN,NaN,1487.425524,1585.689393,NaN,NaN,NaN,NaN,NaN,1485.277608,...,NaN,NaN,1491.681934,NaN,1491.945372,1441.945582,1480.327348,1497.078707,NaN,1468.298904
2019-06-05,NaN,NaN,1473.301284,1580.784232,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1446.916587,NaN,NaN,NaN,1443.751284,1475.091242,NaN,NaN,1450.854843
2020-06-04,NaN,NaN,1489.892882,1574.674949,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1466.854386,NaN,NaN,1430.651881
2021-06-04,NaN,NaN,1482.284277,1564.924107,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
ratingsHistory_df.columns.nunique(),ratingsHistory_df.columns.shape

(32316, (32316,))

In [8]:
# fill in missing values with last seen rating.  
ratingsHistory_df = ratingsHistory_df.ffill(axis=0).fillna(1500)

In [9]:
def get_recent_rating_wp(ratingsHistory_df,tourney_date,winner_id,loser_id):
    # if not type(tourney_date) == pandas._libs.tslibs.timestamps.Timestamp:
    #     tourney_date = datetime.strptime(tourney_date, '%Y-%m-%d').date()

    timestamp = max(ratingsHistory_df.index[ratingsHistory_df.index<=tourney_date])
    winner_rating, loser_rating = ratingsHistory_df.loc[timestamp,[winner_id,loser_id]]
    wp = 1/(1 + 10**((loser_rating-winner_rating)/400))
    return winner_rating, loser_rating, wp

In [10]:
padRow = pd.DataFrame({col: 1500.0 for col in ratingsHistory_df.columns}, index = [pd.Timestamp('1877-07-09T00')])
padRow

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,...,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0,1500.0


In [11]:
ratingsHistory_df.columns.shape

(32316,)

In [12]:
ratingsHistory_df.shape, padRow.shape

((145, 32316), (1, 32316))

In [13]:
padRow.index

DatetimeIndex(['1877-07-09'], dtype='datetime64[ns]', freq=None)

In [14]:
ratingsHistory_df.shape

(145, 32316)

In [15]:
ratingsHistory_df = pd.concat([padRow,ratingsHistory_df],axis=0)

In [16]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1878-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,1500.00000,1502.609587,1487.425524,1585.689393,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1453.542135,1491.681934,1500.00000,1491.945372,1441.945582,1480.327348,1497.078707,1490.832406,1468.298904
2019-06-05,1500.00000,1502.609587,1473.301284,1580.784232,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1475.091242,1497.078707,1490.832406,1450.854843
2020-06-04,1500.00000,1502.609587,1489.892882,1574.674949,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2021-06-04,1500.00000,1502.609587,1482.284277,1564.924107,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881


In [17]:
matches.head(1)

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540


In [18]:
get_recent_rating_wp(ratingsHistory_df,'1877-07-09',113987,114149)

(1500.0, 1500.0, 0.5)

In [19]:
get_recent_rating_wp(ratingsHistory_df,'1878-07-09',113987,114149)

(1532.0, 1508.0, 0.5344839447268315)

In [20]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [21]:
matches

,tourney_date,winner_id,loser_id,tourney_id
0,1877-07-09,113987,114149,1877-540
1,1877-07-09,113987,113999,1877-540
2,1877-07-09,113987,113963,1877-540
3,1877-07-09,113963,114158,1877-540
4,1877-07-09,113999,113953,1877-540
...,...,...,...,...
876973,2022-12-26,207987,208519,2022-M-ITF-TUN-63A-2022
876974,2022-12-26,144932,209977,2022-M-ITF-TUN-63A-2022
876975,2022-12-26,209955,209942,2022-M-ITF-TUN-63A-2022
876976,2022-12-26,209079,210696,2022-M-ITF-TUN-63A-2022


In [22]:
singles_matches.shape,matches.shape

((876978, 49), (876978, 4))

In [23]:
ratingsHistory_df

,131500,131584,131866,131867,131869,131873,131876,131879,131881,131884,...,130340,130355,130457,130493,130542,130550,130552,130553,130655,130767
1877-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1878-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1879-07-09,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1880-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
1881-07-08,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.00000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-06-05,1500.00000,1502.609587,1487.425524,1585.689393,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1453.542135,1491.681934,1500.00000,1491.945372,1441.945582,1480.327348,1497.078707,1490.832406,1468.298904
2019-06-05,1500.00000,1502.609587,1473.301284,1580.784232,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1475.091242,1497.078707,1490.832406,1450.854843
2020-06-04,1500.00000,1502.609587,1489.892882,1574.674949,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881
2021-06-04,1500.00000,1502.609587,1482.284277,1564.924107,1503.281009,1467.575422,1491.136469,1442.915338,1481.686142,1485.277608,...,1490.231718,1446.916587,1491.681934,1500.00000,1491.945372,1443.751284,1466.854386,1497.078707,1490.832406,1430.651881


In [24]:
matches[['tourney_date','winner_id','loser_id']][100:120].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

100                                (1500.0, 1500.0, 0.5)
101                                (1500.0, 1500.0, 0.5)
102                 (1492.0, 1500.0, 0.4884891087822083)
103      (1507.631846603239, 1500.0, 0.5109813439742575)
104                 (1500.0, 1492.0, 0.5115108912177917)
105                                (1500.0, 1500.0, 0.5)
106                                (1500.0, 1500.0, 0.5)
107                 (1492.0, 1500.0, 0.4884891087822083)
108                 (1492.0, 1500.0, 0.4884891087822083)
109                                (1500.0, 1500.0, 0.5)
110                                (1492.0, 1492.0, 0.5)
111     (1492.0, 1500.1841742594847, 0.4882242044507261)
112      (1523.2633029620613, 1492.0, 0.544870470362195)
113                                (1500.0, 1500.0, 0.5)
114     (1491.8158257405153, 1500.0, 0.4882242044507261)
115                                (1500.0, 1500.0, 0.5)
116                                (1500.0, 1500.0, 0.5)
117                 (1492.0, 15

In [37]:
matches.dtypes

tourney_date    datetime64[ns]
winner_id                int64
loser_id                 int64
tourney_id              object
dtype: object

In [36]:
testDf = singles_matches[0:10]
testDf[['winner_elo','loser_elo','predicted_wins']] = matches[0:10].apply(lambda x: 
get_recent_rating_wp(matches[0:10], x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)
testDf[0:1]

TypeError: '<=' not supported between instances of 'numpy.ndarray' and 'Timestamp'

In [28]:
singles_matches[['winner_elo','loser_elo','predicted_wins']] = matches[['tourney_date','winner_id','loser_id']].apply(lambda x: 
get_recent_rating_wp(ratingsHistory_df, x['tourney_date'], x['winner_id'], x['loser_id']),axis=1)

KeyboardInterrupt: 

In [ ]:
ratingsHistory_df